In [1]:
import argparse
from apiclient.discovery import build
import httplib2
from oauth2client import client
from oauth2client import file
from oauth2client import tools
import re
import requests
import pandas as pd
import numpy as np
import openpyxl

import os 
from datetime import datetime, timedelta
import pandas as pd 
import dataframe_image as dfi
import plotly.graph_objects as go
import plotly.offline as offline
import time
import pdfkit

from Jira_Data import getTickets, download_attachments

from Significance_Calculation import typeofExperiment
import warnings
warnings.filterwarnings('ignore')

from bellaandDuke import bellaAndDuke
from falvourly import flavourly
from moss import moss
from moda import moda

from Revenue_Forecast_v2 import revenue_forecast, create_graph

In [2]:
from chums import chums
from blindsDirect import blindsDirect
from polesDirect import polesDirect

In [3]:
import smtplib
from email.mime.text import MIMEText
from email.header    import Header
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.utils import formatdate
from email import encoders
from email.mime.application import MIMEApplication

import sys

In [4]:
def missingDataTicket(ticket):
    missingList = []
    singleData, multiDevice, noData,data = typeofExperiment(ticket)
    if noData == True:
        missingList.append('No Data to Process')
        

    if ticket['Hypothesis 1 We Observed (Evidence, Feedback)'] is None:
        missingList.append('Hypothesis 1')
    if ticket['Hypothesis 2 We Believe (Change, Groups, Outcomes)'] is None:
        missingList.append('Hypothesis 2')
    if ticket['Hypothesis 3 We will Know When this Happens (Data, Feedback)'] is None:
        missingList.append('Hypothesis 3')
        
    if data is not None: 
        if data['controlId'] is None:
            missingList.append('Control ID')
        if data['variationId'] is None:
            missingList.append('Variation ID')

    
    if (ticket['Desktop Variation(s) Description'] is None) and ('Desktop' in ticket['Devices']):
        missingList.append('Desktop Variation(s) Description')
    
    if (ticket['Tablet Variation(s) Description'] is None) and ('Tablet' in ticket['Devices']):
        missingList.append('Tablet Variation(s) Description')
    
    if (ticket['Mobile Variation(s) Description'] is None) and ('Mobile' in ticket['Devices']):
        missingList.append('Mobile Variation(s) Description')
    

    if ticket['Psychological Principles'] is None:
        if ticket['Psychological Principles/Effects'] is None:
            missingList.append('Psychological Principles')
    elif ticket['Psychological Principles'][0]   == 'Other':
        if ticket['Psychological Principles/Effects'] is None:
            missingList.append('Psychological Principles')
    
    return missingList


In [23]:
tickets_df_mapped = getTickets(status="Ended\ -\ Reporting")

In [24]:
tickets_df_mapped['desktoptestTitle'] = np.where(tickets_df_mapped['Client'] == 'Blinds Direct',tickets_df_mapped['Title'],tickets_df_mapped['desktoptestTitle'])

In [25]:
import jira as JIRA

In [26]:
import os 
from Jira_Data import getTickets, download_attachments

In [27]:

from jira import JIRA
import re
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import smtplib
from email.mime.text import MIMEText
from email.header import Header
from datetime import date


def download_attachments(issueId):
    '''
    Parameters : issueId 

    This module will download all the available attachments in the given JIRA ID 

    Note: The issueId is the Jira ticket number NOT THE EXPERIMENT NUMBER 
    '''
    options = {'server': "https://endlessgain.atlassian.net"}
    jira = JIRA(options, basic_auth=('pooja.bhat@endlessgain.com', 'WSeX2dWnPw8NyoJqNUOP7BDE'))

    jira_issue = jira.issue(issueId, expand="attachment")

    attachment_list = []

    for attachment in  jira_issue.fields.attachment  :    
        jira_filename = attachment.filename    
        attachment_list.append(jira_filename)
    
    return attachment_list


In [28]:
a = []
for i in range(len(tickets_df_mapped)):
    ticket = tickets_df_mapped.iloc[i]
#     os.mkdir(f'./Misc/Data/{ticket['JIRA_ID']}')
#     attachments = download_attachments(ticket['JIRA_ID'])
    images_missing = ''
    len_val = 0
    if ticket['desktopvariationIds'] is not None:
        len_val = len(ticket['desktopvariationIds'].split(','))
    if ticket['mobilevariationIds'] is not None:
        len_val = len(ticket['mobilevariationIds'].split(','))
    if ticket['tabletvariationIds'] is not None:
        len_val = len(ticket['tabletvariationIds'].split(','))


    files =  download_attachments(ticket['Keys'])
    for i in ticket['Devices']:
        if len([ a for a in files if a.startswith(i.lower())]) < len_val:
            images_missing = 'Images Missing'
    
    
    a.append([ticket['Client'],ticket['Keys'], ticket['Title'],missingDataTicket(ticket),images_missing])
    
    print(ticket['Client'],ticket['Keys'],images_missing)
    
    

Moda MODTD-88 
Moda MODTD-74 
Moda MODTD-73 
Moda MODTD-72 
Moda MODTD-71 Images Missing
Moda MODTD-70 
Moda MODTD-69 Images Missing
Moda MODTD-68 Images Missing
Moda MODTD-56 Images Missing
Moda MODTD-52 
Instasmile UK INST-4 
Instasmile US INST-4 
Hotter HOTTD-242 
Hotter HOTTD-236 
Hotter HOTTD-234 
Hotter HOTTD-233 
Hotter HOTTD-232 
Hotter HOTTD-229 
Hotter HOTTD-225 
Flavourly FLTD-26 
Flavourly FLTD-25 
Flavourly FLTD-24 
Flavourly FLTD-23 
Flavourly FLTD-19 
Flavourly FLTD-15 
Flavourly FLTD-13 
Flavourly FLTD-12 
Studio EXGTD-104 
Studio EXGTD-92 
Studio EXGTD-90 
Studio EXGTD-86 
Studio EXGTD-81 Images Missing
Chums CHUMTD-69 
Cadogan CCTD-39 
Cadogan CCTD-34 
Cadogan CCTD-29 
Cadogan CCTD-28 
Cadogan CCTD-26 
Cadogan CCTD-25 
Cadogan CCTD-12 
Bella & Duke BLDU-17 
Bella & Duke BLDU-15 
Bella & Duke BLDU-11 
Bella & Duke BLDU-10 
Bella & Duke BLDU-9 
Bella & Duke BLDU-6 Images Missing
Bella & Duke BLDU-5 
Bella & Duke BLDU-3 Images Missing
Bella & Duke BLDU-2 Images Missing
B

In [29]:
ticket = tickets_df_mapped.iloc[19]

In [30]:
df = pd.DataFrame(a)

In [31]:
df.shape

(57, 5)

In [32]:
 df['Missing_Length'] = df[3].apply(len)

In [33]:
df

,0,1,2,3,4,Missing_Length
0,Moda,MODTD-88,MO 057 - Link To Indoor Furniture,[],,0
1,Moda,MODTD-74,MO 047 - Text Highlighting Message,[],,0
2,Moda,MODTD-73,MO 046 - Returning users basket popup,[],,0
3,Moda,MODTD-72,MO 045 - Product Recommendations,[],,0
4,Moda,MODTD-71,MO 044 - Custom Range notification,[],Images Missing,0
5,Moda,MODTD-70,MO 041 - Recently Viewed Items,[],,0
6,Moda,MODTD-69,MO 042 - Dynamic tab text,[],Images Missing,0
7,Moda,MODTD-68,MO 040 - Exit Detection Email Collection,[],Images Missing,0
8,Moda,MODTD-56,MO 028 Rebrand - Weather Inspiration - Homepage,[],Images Missing,0
9,Moda,MODTD-52,MO 032 - Add to Basket Upsell - PDP,[],,0


In [34]:
df = df[(df['Missing_Length'] > 0) | (df[4] == 'Images Missing')]

In [35]:
df = df.rename({0:'Client',1:'Jira Ticket', 2:'Experiment Name', 3:'Missing Fields', 4:'Images Missing'}, axis =1 )

In [36]:
df = df[~df['Experiment Name'].str.startswith('ROLLOUT')]

In [37]:
df.to_csv('Missing_Ended_10_May.csv', index=False)

In [38]:
df.shape

(22, 6)